In [2]:
# ! pip install fitz
! pip install pymupdf

  Using cached PyMuPDF-1.24.9-cp310-none-win_amd64.whl.metadata (3.4 kB)
  Using cached PyMuPDFb-1.24.9-py3-none-win_amd64.whl.metadata (1.4 kB)
Using cached PyMuPDF-1.24.9-cp310-none-win_amd64.whl (3.2 MB)
Using cached PyMuPDFb-1.24.9-py3-none-win_amd64.whl (13.2 MB)


In [3]:
import fitz

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = []
    for page in doc:
        text.append(page.get_text())
    return "\n".join(text)

pdf_text = extract_text_from_pdf("ex1.pdf")

Task 4:  Counting the Number of Words per Character in the Script

In [5]:
import re
from collections import defaultdict
import pandas as pd

In [16]:
## method 1: using regex
# we are identifying character names by if they are all caps and on a single line
# we do exclude lines with scene descriptors like int and ext to not pick up on setting all caps lines
# Problems: the scripts are formatted differently and this does not pick up on the line "DADDY and MAMA" for ex because "and" is lowercase

def count_words_per_character(script_text):
    lines = script_text.split('\n')
    word_counts = defaultdict(int)
    current_characters = []

    # Identifying character names as all caps on a single line, including commas and ampersands
    pattern = re.compile(r'^[A-Z\s,&]+$')

    for line in lines:
        line = line.strip()
        match = pattern.match(line)
        if match:
            # Skip common scene description indicators to prevent matching with all caps setting
            if any(word in line for word in ['INT', 'EXT', 'DAY', 'NIGHT', 'MORNING', 'AFTERNOON', 'EVENING']):
                current_characters = []
            else:
                # Split the line by commas, ampersands, and ANDs to get individual character names
                characters = [char.strip() for char in re.split('[,&]| AND | and ', line)]
                current_characters = characters
                # if len(current_characters)>1:
                #     print(current_characters)
        elif current_characters:
            # Count words in the dialogue lines
            word_count = len(line.split())
            for char in current_characters:
                # if char == "MAMA":
                #     print("mama", word_count)
                word_counts[char] += word_count

    return word_counts

# Count words per character
word_counts = count_words_per_character(pdf_text)

# Display the results
df = pd.DataFrame(word_counts.items(), columns=["Character", "Word Count"])
print(df)

       Character  Word Count
0          NAMIA         573
1           NUZO         730
2          DADDY         138
3           MAMA           7
4       BUBELANG         738
5  FEMALE SELLER         113


In [17]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

# Load the pre-trained model and tokenizer
model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create the NER pipeline
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ModuleNotFoundError: No module named 'transformers'